# Table of Contents

1.  [Experience](#org3f6a792)
2.  [Haskelisms](#org6511997)
    1.  [Strict vs Lazy (Wrong Question)](#org6e94ec9)
    2.  [Strictness](#org5a17db8)
    3.  [Non-strictness](#org7df1c52)
        1.  [A `non-strict` language evaluates from the outside in](#org9009907)
        2.  [Means a function will NOT always evaluate its second argument.](#orgf67692b)
    4.  [Monad Transformers](#org05a8ee5)
        1.  [A technique designed for a non-strict, lazy by default environment](#orgddeb051)
        2.  [Functor & Monad](#orgabba8d1)
        3.  [Monad Transformer shape](#org8d25129)
        4.  [All together](#org4439575)
    5.  [Monads & Trampolining](#org28e2205)
        1.  [Trampolines trade stack for heap](#orgc0cedda)
        2.  [Stackless Scala with Free Monads](#orgd4720dc)
3. [Tagless Final](#taglessfinal)
3. [In Summary](#summary)

<a id="org3f6a792"></a>

# Experience

or context matters
or past experience shapes current opinion
or why I care about ZIO

* Eniro Initiatives – Mañana Data Processing Hybrid Cloud Solution (Aug. 2018 – Present)
* Carrefour – ATUIN Order Management Backend (Sept. 2017 – Jul. 2018)
* El Corte Inglés – Data-centric Marketplace Catalog (Oct. 2016 – Jul. 2017)
* El Corte Inglés – Customer Service Center (CSC), Omnistore, Multi-Channel platform (Dec. 2013 – Sep. 2016)
* Jazztel – Online ATG Store (Apr. 2013 – Aug. 2013)
* Louis Vuitton Moet Hennessy – Conducting technical audit (Feb. 2013 – Mar. 2013)
* Philips – Lighting B2B (Nov. 2012 – Mar. 2013)
* El Corte Inglés – Multiorder module and appliances store (Apr. 2010 – Oct. 2012)
* CSIC – Intranet and Web Portal (May 2009 – Mar. 2010)

aka: Very ENTERPRISEY!


<a id="org6511997"></a>

# Haskelisms

****Avoid applying patterns designed for non-strict and lazy-by-default execution environments on strict and eager ones****


<a id="org6e94ec9"></a>

## Strict vs Lazy (Wrong Question)

* Stict vs Non-strict => Denotational Semantics - Reduction stage (What will be evaluated)
* Eager vs Lazy => Operational Semantics - Evaluation stage (How will it be evaluated)
* Strict & Eager => C, Python, Ruby, Go, JVM (and all its languages)
* Strict & Lazy => Scala can behave as with Effect Systems
* Non-strict & Lazy => Haskell
* Non-stric & Eager => Haskell can behave as by evaluating to
* [Lazy vs non-strict](https://wiki.haskell.org/Lazy_vs._non-strict)


<a id="org5a17db8"></a>

## Strictness

In haskell, its used to mean that a function evaluates its argument to [WHNF](20200512103249-whnf.md) before proceeding.
While `(square 42)` can be reduced to `(42 * 42)`, which can itself be reduced to a ****normal form**** of `1764`, `Just (square 42)` is WHNF without further evaluation.


<a id="org7df1c52"></a>

## Non-strictness


<a id="org9009907"></a>

### A `non-strict` language evaluates from the outside in

Given:

-   a    = 5
-   b    = 50
-   c    = 3
-   f(x) = x + 5
-   g(x) = 2\*x

// non-strict  
(f(a) + g(b*c))  
(f(5) + g(b*c))  
(10 + g(b*c)) // can't proceed untl evaluating right side  
(10 + g(50*3))  
(10 + g(150))  
(10 + 300)  
(310)  
    
// strict  
(f(a) + g(b*c))  
(f(5) + g(b*c))  
(f(5) + g(50*3))  
(f(5) + g(150)  
(10 + g(150))  
(10 + 300)  
(310)  

<a id="orgf67692b"></a>

### Means a function will NOT always evaluate its second argument.


In [ ]:
def maybeExplode(b: Boolean): Int = {
  val tup = (0, ???)
  if (b) tup._1
    else tup._2
}
    
maybeExplode(true)

-   A Strict language will explode (throw `NotImplementedError`)
-   A Non-strict language will return **0** on **true** and explode otherwise.


<a id="org05a8ee5"></a>

## Monad Transformers


<a id="orgddeb051"></a>

### A technique designed for a non-strict, lazy by default environment

1.  What happens when applied on the JVM?

    1.  We use `thunks` to simulate lazyness by delaying execution until it is needed (call by-name)

In [ ]:
def doSomeThing(op: => Unit) {
    println("thunking...")
    op
}

doSomeThing(println("Hello!"))

def doOtherThing(op: () => Unit) {
    println("thunking too...")
    op()
}
doOtherThing(() => println("Hello!"))


    2.  But we can't simulate non-strict semantics


<a id="orgabba8d1"></a>

### Functor & Monad

Usually you define `Functor` for its `map` method but for briefness, let&rsquo;s write it directly in `Monad`

In [ ]:
trait Monad[F[_]] {
    
  def pure[A](a: A): F[A]
  def flatMap[A, B](fa: F[A])(f: A => F[B]): F[B]

  def map[A, B](fa: F[A])(f: A => B): F[B] = flatMap(fa)(a => pure(f(a)))
}

object Monad {

  def apply[F[_]](implicit m: Monad[F]): Monad[F] = m

  def flatMap[F[_]: Monad, A, B](fa: F[A])(f: A => F[B]) =
    Monad[F].flatMap[A, B](fa)(f)

  def pure[F[_]: Monad, A](a: A): F[A] = Monad[F].pure[A](a)

  implicit class MonadOps[F[_]: Monad, A](fa: F[A]) {
    def flatMap[B](f: A => F[B]) = Monad[F].flatMap[A, B](fa)(f)
  }

  implicit val optionMonad: Monad[Option] = new Monad[Option] {
    def pure[A](a: A): Option[A] = Option(a)

    // def flatMap[A, B](fa: Option[A])(f: A => Option[B]): Option[B] = fa.flatMap(f)
    def flatMap[A, B](fa: Option[A])(f: A => Option[B]): Option[B] = fa match {
      case None    => None
      case Some(a) => f(a)
    }
  }

  implicit val listMonad: Monad[List] = new Monad[List] {

    override def pure[A](a: A): List[A] = List(a)

    override def flatMap[A, B](fa: List[A])(f: A => List[B]): List[B] =
      fa match {
        case Nil     => List.empty[B]
        case x :: xs => f(x) ++ flatMap(xs)(f)
      }
  }
}

<a id="org8d25129"></a>

### Monad Transformer shape

In [ ]:
trait TransM[F[_[_], _]] {
  def lift[G[_]: Monad, A](a: G[A]): F[G, A]
}

case class OptionT[F[_]: Monad, A](value: F[Option[A]]) {

  // Applicative is preferred
  def apply(a: A): OptionT[F, A] =
    OptionT(Monad[F].pure(Some(a)))

  def pure(a: A): OptionT[F, A] = apply(a)

  def flatMap[B](f: A => OptionT[F, B]): OptionT[F, B] =
    OptionT(
      Monad[F]
        .flatMap(value)(_.fold(Monad[F].pure[Option[B]](None))(a => f(a).value))
    )

  def map[B](f: A => B): OptionT[F, B] =
    OptionT(Monad[F].map(value)(_.map(f)))
}

case class ListT[F[_]: Monad, A](value: F[List[A]]) {

  // Applicative is preferred
  def apply(a: A): ListT[F, A] =
    ListT(Monad[F].pure(List(a)))

  def pure(a: A): ListT[F, A] = apply(a)

  def flatMap[B](f: A => ListT[F, B]): ListT[F, B] =
    ListT(
      Monad[F].flatMap(value) {
        _ match {
          case Nil => Monad[F].pure(List.empty[B])
          case lst => lst.map(f).reduce(_ ++ _).value
        }
      }
    )

  def map[B](f: A => B): ListT[F, B] =
    ListT(Monad[F].map(value)(_.map(f)))

  def ++(that: ListT[F, A]) = ListT(
    Monad[F].flatMap(value) { lst => Monad[F].map(that.value)(lst ++ _) }
  )
}

object TransM {
  implicit val optionTransM: TransM[OptionT] = new TransM[OptionT] {
    // Functor is peferred
    override def lift[G[_], A](ga: G[A])(implicit G: Monad[G]): OptionT[G, A] =
      OptionT(G.map(ga)(Some(_)))
  }

  implicit val listTransM: TransM[ListT] = new TransM[ListT] {
    // Functor is peferred
    override def lift[G[_], A](ga: G[A])(implicit G: Monad[G]): ListT[G, A] =
      ListT(G.map(ga)(List(_)))
  }
}

<a id="org4439575"></a>

### All together

In [ ]:
val l1 = List(1,2,3,4,5)
def calcErr(base: Int, threshold: Int): List[Option[Int]] = for {
    i   <- l1
    opt <- toOption(i, (i => (i + base) <= threshold))
} yield  opt

println(calcErr(6, 12))
/*
Fails with:
found   : List[Int]
required: List[Option[Int]] (lsp)
*/

We use Monad Transformers to have everything under the same monad

In [ ]:
type ListOption[A] = OptionT[List, A]
    
val lst1 = List(Option(1), Option(2), Option(3), Option(4), Option(5))
val lst2 = List(Option(1), Option(2), Option(3), None, Option(5))
val r1: ListOption[Int] = OptionT(lst1)
val r2: ListOption[Int] = OptionT(lst2)
import TransM._
val ra: ListOption[Int] = optionTransM.lift(List(6))

val calc: ListOption[Int] => OptionT[List, Int] = (r: ListOption[Int]) =>
  for {
    l1 <- r
    l2 <- ra
  } yield l1 + l2

println(calc(r1))
println(calc(r2))

1.  How does strictness/non-strictness affect type inference?

2.  How does it affect performance?

In [ ]:
val l = (1 to 11000).map(Option(_)) // Could `map` even numbers to `None`
println(calc(OptionT(l.toList)).value.size)

Scala lists have optimizations:

In [ ]:
val l = (1 to 30000).map(Option(_))
val t1 = l.foldLeft(OptionT(List(Option(0))))((acc, c) => {
  val lst = acc.value
  val j = c.fold(0)(_ + lst.head.getOrElse(0))
  OptionT(Option(j) +: lst)
})
println(t1.value.filter(_ != Some(0)).length)

But that&rsquo;s not the case for our custom ADTs


<a id="org28e2205"></a>

## Monads & Trampolining


<a id="orgc0cedda"></a>

### Trampolines trade stack for heap


<a id="orgd4720dc"></a>

### [Stackless Scala with Free Monads](http://blog.higher-order.com/assets/trampolines.pdf)

-   When  the  compiler  finds  that  ****a  method  calls  itself  in the  tail  position****,  and  if  that ****method  cannot  be overridden**** (e.g.  because  of  being ****declared private or final****),  therecursive method invocation is replaced with a simple jumping the compiled code. This is ****equivalent to rewriting the tail-recursion as a loop****.
-   While optimizing self-recursive calls is easy, replacing tail calls in general with jumps is more difficult. Currently,  the  Java  virtual  machine  (JVM)  allows  only  local jumps, so there is no way to directly implement a tail call to another method as a jump.

1.  Usage

    -   Scalaz offers trampolines as part of `Free` AND `tailRecM` via [BindRec](https://github.com/scalaz/scalaz/blob/867ea5466b470e53679bb7aeb054165b9c2d9176/core/src/main/scala/scalaz/BindRec.scala)
    -   Cats exposes `tailRecM` as a way to ensure stack safety
    -   ZIO uses trampolining, for instance, in `FiberContext`

2.  Implementation

In [ ]:
import annotation.tailrec
        
trait Trampoline[+A] {
 @tailrec
 final def resume: Either[() => Trampoline[A], A] = this match {
    case Done(v) => Right(v)
    case More(k) => Left(k)
    case FlatMap(a, f) =>
      a match {
        case Done(v)       => f(v).resume
        case More(k)       => Left(() => k() flatMap f)
        case FlatMap(b, g) => b.flatMap((x: Any) => g(x) flatMap f).resume
      }
  }

  // should advance to next step?
  final def runT: A =
    resume match {
      case Right(a) => a
      case Left(k)  => k().runT
    }

  final def flatMap[B](f: A => Trampoline[B]): Trampoline[B] =
    /*this match {
      case FlatMap(a, g) => FlatMap(a, (x: Any) => g(x) flatMap f)
      case x             => FlatMap(x, f)
     }*/
    FlatMap(this, f)

  final def map[B](f: A => B): Trampoline[B] =
    flatMap(a => Done(f(a)))
}

case class Done[+A](result: A) extends Trampoline[A]

case class More[+A](k: () => Trampoline[A]) extends Trampoline[A]

private case class FlatMap[A, +B](sub: Trampoline[A], k: A => Trampoline[B]) extends Trampoline[B]

3.  In Action

In [ ]:
type ListTramp[A] = ListT[Trampoline, A]
type ListOption[A] = OptionT[ListTramp, A]

val lst1: ListTramp[Int] = ListT(More(() => Done(List(1))))

/*
Fails with:
could not find implicit value for evidence parameter of type es.hybride.Monad[es.hybride.More]

could not find implicit value for evidence parameter of type es.hybride.Monad[[A]es.hybride.ListT[es.hybride.Trampoline, A]]
*/

1.  We need more Monads!

In [ ]:
type ListTramp[A] = ListT[Trampoline, A]
type ListOption[A] = OptionT[ListTramp, A]

implicit val trampolineMonad = new Monad[Trampoline] {
    override def pure[A](a: A): Trampoline[A] = More(() => Done(a))

    override def flatMap[A, B](fa: Trampoline[A])(f: A => Trampoline[B]): Trampoline[B] =
      fa.flatMap(f)

    override def map[A, B](fa: Trampoline[A])(f: A => B): Trampoline[B] =
      fa.map(f)
}

implicit def listTMonad[F[_]: Monad, A] = new Monad[({type T[A] = ListT[F, A]})#T] {
    override def pure[A](a: A): ListT[F, A] = ListT(Monad[F].pure(List(a)))

    override def flatMap[A, B](fa: ListT[F,A])(f: A => ListT[F,B]): ListT[F,B] =
      fa.flatMap(f)

    override def map[A, B](fa: ListT[F,A])(f: A => B): ListT[F,B] =
      fa.map(f)
}

val lst1: ListTramp[Int] = ListT(Monad[Trampoline].pure(List(1)))
val lsta: ListTramp[Int] = listTransM.lift(trampolineMonad.pure(6))

val ra: ListOption[Int] = OptionT(lsta.map(i => Option(i)))
val r1: ListOption[Int] = OptionT(lst1.map(i => Option(i)))

val calc: ListOption[Int] => OptionT[ListTramp, Int] = (r: ListOption[Int]) =>
  for {
    l1 <- r
    l2 <- ra
  } yield l1 + l2

println(calc(r1).value.value.runT.size)

And finally:

In [ ]:
val l = ListT(Monad[Trampoline].pure((1 to 30000).toList))
            val r = OptionT(l.map(i => Option(i)))
            println(calc(r).value.value.runT.size)

Its not uncommon to have a 

```scala
Either[String, HttpRequest[DBRequest[List[Option[User]]]]]
```  
<a id="taglessfinal"></a>

# Tagless Final

1. Declare some Algebras

```scala  
trait EventSourcingAlgebra[F[_]] {
  def parseMessage(msg: String): F[Option[Json]]
  def handleCommand(json: Json): F[Option[Input]]
}

trait MessagingAlgebra[F[_]] {
  def receiveMessage(brokers: String, topic: String, consumerGroup: String, autoCommit: Boolean): F[Option[String]]
  def commit(): F[Unit]
  def sendMessage(brokers: String, topic: String, message: String): F[String]
  def replay(id: Id, entity: ReplayMsg): F[String]
}

trait OrdersAlgebra[F[_]] {
  def createOrder(id: Id, entity: JsonOrder): F[OrderEvent[Order]]
  def addCommerceItem(orderId: Id, entity: SubProduct, product: Product, qty: Int): F[OrderEvent[CommerceItem]]
  def addPaymentMethod(orderId: Id, entity: PaymentMethod): F[OrderEvent[PaymentGroup]]
  def addPaymentAddress(orderId: Id, entity: Address): F[OrderEvent[PaymentGroup]]
  def unknownCommand(id: Id): F[OrderEvent[Order]]
}
```  

2. Write interpreters  

```scala  
trait EventSourceInterpreter extends EventSourcingAlgebra[Future] { ... }
trait MessagingInterpreter extends MessagingAlgebra[Future] { ... }
trait OrdersInterpreter extends OrdersAlgebra[Future] { ... }
```  

3. Write your program

```scala  
class Programs[F[_] : Monad](msgCtx: MessagingAlgebra[F], ordersCtx: OrdersAlgebra[F], esCtx: EventSourcingAlgebra[F]) {

  def processMessage(brokers: String, topic: String, consumerGroup: String, autoCommit: Boolean)
                    (implicit eventLog: EventStore[String]): F[Stream[String]] =
    for {
      message <- msgCtx.receiveMessage(brokers, topic, consumerGroup, autoCommit)
      json <- esCtx.parseMessage(message.getOrElse(""))
      key = json.flatMap(j => (j \\ "key").head.asString)
      entity <- esCtx.handleCommand(json.getOrElse({}.asJson))
      eventStream <- entity match {
        case Some(e) => e match {
          case order@JsonOrder(id, _, _) => ordersCtx.createOrder(key.getOrElse(id), order).map(Stream(_))
          case address@Address(id, _, _) => ordersCtx.addPaymentAddress(key.getOrElse(id), address).map(Stream(_))
          ...
          case _ => implicitly[Monad[F]].pure(failedEvent("Unknown command")).map(Stream(_))
        }
        case None => implicitly[Monad[F]].pure(failedEvent("Unknown command")).map(Stream(_))
      }
      out <- {
        val res: Stream[F[String]] = eventStream.map(event => msgCtx.sendMessage(brokers, topic, event.projection.asJson.noSpaces))
        res.streamSequence
      }
    } yield out
}
```  

4. Provide your interpreters to your program

```scala  
class MultiInterpreters(val eventStore: EventStore[String]) {

  val futureMessagingInterpreter = new MessagingInterpreter {}
  val futureOrdersInterpreter = new OrdersInterpreter { ... }
  val futureeEventSourcingInterpreter = new EventSourceInterpreter {}

  def run(): Unit = {
    executor.execute(() => {
      println(s"Interpreter executing on thread ${Thread.currentThread().getName} ${Thread.currentThread().getId}")
      while (true) {
        val result: Future[Stream[String]] = new Programs(futureMessagingInterpreter, futureOrdersInterpreter, futureeEventSourcingInterpreter)
          .processMessage("192.168.99.100:9092", "test", "testers", false)(eventStore)

        result.filter(_.nonEmpty).foreach(s => println(s"message processed: $s"))
      }
    })
  }
}
```  

* The key idea is that all implementations return the same `F[_]: Monad` so you don't need transformers.
* In real life, however, you may still need to return a `List[Either[String,A]]` or something, so transformers usage is reduced, but not necessarily eliminated.
* It still needs heavy framework and cognitive machinery.

Can play with the code on https://github.com/toxicafunk/ESInterpreters/tree/finally_tagless


<a id="summary"></a>

# In Summary

* Scala is a great flexible language with a *Type System* so powerful it allows us to write in almost any style we want or need (OO, FP, Actors, FRP, Co-routines, Declarative/DSLs, etc.)
* The fact that we can write Pure FP in the style of Haskell is a testament to the versatility of the language and the creativity and inventiveness of its community.
* But it comes at a cost... techniques like Trampolines or the use of Context Bounds in TF improves performance  but don't eliminate the cost.
* Scala can benefit from **Scalaisms**  - a way to write FP that takes advantages of Scala's strengths instead of working agains idiomatic Scala.